## LIBRERIAS y CONSTANTES

In [5]:
import numpy as np
import pandas as pd
import pickle
import torch.nn as nn
from torch.utils.data import DataLoader
import json, ast, sys, csv, random
import plotly.express as px
import math
import datetime
import sys
import csv

#Implement training process
from model_trees_algebra import NeoRegression

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from functions.tree_format import IterateBuildTree, InnerJoinsIntraBGPS, \
                                IterateBuildTreeBetweenBGPS, TreeFormat, TreeFormat_all
from functions.RL_functions import GetTriplesSubtree, \
                                        GetTreeSize, \
                                        GetAllJoins, \
                                        GetIter, \
                                        GetTotalBgp, \
                                        GetDataframe

from functions.aux import MetricTotalAccuraccy


class BaoTrainingException(Exception):
    pass
csv.field_size_limit(sys.maxsize)
URL = "/media/data/ccarmona/memoria/dataset/"

In [7]:
df_train = pd.read_csv(URL + "train_old_dataset_4.2_subqueries.csv", engine='python', encoding='utf-8')
df_test = pd.read_csv(URL + "test_old_dataset_4.2_subqueries.csv", engine='python', encoding='utf-8')

In [18]:
def GetTreeSize(subtrees, treesize):
    if len(subtrees) == 1:
        return treesize
    else:
        treesize += 1
        left_treesize = GetTreeSize(subtrees[1], treesize)
        right_treesize = GetTreeSize(subtrees[2], treesize)
        treesize = max(left_treesize,right_treesize)
    return treesize

In [10]:
for i in df_train.columns:
    print(i)
print(df_test.columns == df_train.columns)

unique_id
filename
query
profile
limit
group_by
distinct
order_by
union
left_join
join
iter
filter
num_filter
filter_eq
filter_gt
filter_ge
filter_lt
filter_le
filter_neq
filter_iri
filter_neq.1
filter_bound
filter_contains
filter_exists
filter_isBlank
filter_isIRI
filter_isLiteral
filter_lang
filter_langMatches
filter_not
filter_notexists
filter_regex
filter_sameTerm
filter_str
filter_strstarts
filter_or
filter_and
time
cpu_p
rnd
seq
same_seg_p
same_page_p
disk_reads
read_ahead
wait
comp_msec
comp_reads
comp_read_p
comp_messages
comp_clw
id_old
query_old
time_old
assign_old
bgp_old
distinct_old
extend_old
filter_old
filter_bound_old
filter_contains_old
filter_eq_old
filter_exists_old
filter_ge_old
filter_gt_old
filter_isBlank_old
filter_isIRI_old
filter_isLiteral_old
filter_lang_old
filter_langMatches_old
filter_le_old
filter_lt_old
filter_ne_old
filter_not_old
filter_notexists_old
filter_or_old
filter_regex_old
filter_sameTerm_old
filter_str_old
filter_strends_old
filter_strstarts_ol

In [17]:
df_test['trees'][0]

'["JOINᶲIRI_ID;...direct/P2604;", ["VAR_URI_VARᶲIRI_ID;...direct/P2604;"], ["VAR_VAR_VARᶲNONE"]]'

In [16]:
df_test['trees_old'][0]

'[ "JOINá¶²http://www.wikidata.org/prop/direct/P2604" ,   [ "VAR_URI_VARá¶²http://www.wikidata.org/prop/direct/P2604" ] ,   [ "VAR_VAR_VARá¶²NONE" ] ] '

In [34]:
x1 = ast.literal_eval(df_test['trees'][5])

In [35]:
x2 = ast.literal_eval(df_test['trees_old'][5].replace("á¶²","ᶲ"))

In [36]:
GetTreeSize(x1, 1)

3

In [37]:
GetTreeSize(x2, 1)

3

In [38]:
df_test.shape

(3480, 151)

In [92]:
equal_size = []
not_equal_size = []
not_equal_size_idx_test = []
not_equal_size_idx_train = []
error = []
for i in range(df_test.shape[0]):
    try:
        x1 = ast.literal_eval(df_test['trees'][i])
        x2 = ast.literal_eval(df_test['trees_old'][i].replace("á¶²","ᶲ"))
        size_1 = GetTreeSize(x1, 1)
        size_2 = GetTreeSize(x2, 1)
        if size_1 == size_2:
            equal_size.append((i,"test"))
        else:
            not_equal_size.append((i,"test"))
            not_equal_size_idx_test.append(i)
    except:
        error.append((i,"test"))
        
for i in range(df_train.shape[0]):
    try:
        x1 = ast.literal_eval(df_train['trees'][i])
        x2 = ast.literal_eval(df_train['trees_old'][i].replace("á¶²","ᶲ"))
        size_1 = GetTreeSize(x1, 1)
        size_2 = GetTreeSize(x2, 1)
        if size_1 == size_2:
            equal_size.append((i,"train"))
        else:
            not_equal_size.append((i,"train"))
            not_equal_size_idx_train.append(i)
    except:
        error.append((i,"train"))
                
        
        

In [93]:
print(len(equal_size))
print(len(not_equal_size))
print(len(error))
print(len(not_equal_size_idx_test))
print(len(not_equal_size_idx_train))

26570
2329
7
274
2055


In [102]:
no_cuenta = 0
for i in not_equal_size_idx_test:
    
    if df_test['query'][i].count("*") < 1:
        print(df_test['query'][i])
        print("-------------------")
        print(df_test['trees'][i])
        print("----------------")
        print(df_test['trees_old'][i])
        print("----------------------")
        print("+++++++++++++++++++++")
    else:
        no_cuenta += 1
    
    


PREFIX bds: <http://www.bigdata.com/rdf/search#>
PREFIX gas: <http://www.bigdata.com/rdf/gas#>
PREFIX hint: <http://www.bigdata.com/queryHints#>
PREFIX bd: <http://www.bigdata.com/rdf#>
PREFIX cc: <http://creativecommons.org/ns#>
PREFIX dct: <http://purl.org/dc/terms/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX ontolex: <http://www.w3.org/ns/lemon/ontolex#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX p: <http://www.wikidata.org/prop/>
PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
PREFIX pqn: <http://www.wikidata.org/prop/qualifier/value-normalized/>
PREFIX pqv: <http://www.wikidata.org/prop/qualifier/value/>
PREFIX pr: <http://www.wikidata.org/prop/reference/>
PREFIX prn: <http://www.wikidata.org/prop/reference/value-normalized/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX prv: <http://www.wikidata.org/prop/reference/value/>
PREFIX ps: <http://www.wikidata.org/prop/statement/>
PREFIX psn: <http://www.wikidata.org/prop/statement/value-normalized/>
PRE

In [101]:
print(no_cuenta)

196


In [86]:
ast.literal_eval(df_train['trees'][8984])

['JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direct/P577;ᶲIRI_ID;...description;',
 ['JOINᶲIRI_ID;...direct/P31;ᶲIRI_ID;...direct/P577;',
  ['VAR_URI_URIᶲIRI_ID;...direct/P31;'],
  ['VAR_URI_VARᶲIRI_ID;...direct/P577;']],
 'VAR_URI_VARᶲIRI_ID;...description;']

In [87]:
ast.literal_eval(df_train['trees_old'][8984])

['VAR_URI_URIá¶²http://www.wikidata.org/prop/direct/P31']